*To do*
1. In different years electric cars are counted differently: in early years not at all; in middle years as one category; in newest years in separate hybrid + BEV columns. Electric cars are not currently in the df because this is complicated to program lol

In [2]:
import pandas as pd

## Making PKW dataframes from the years where we have a CSV

This is for all years 2013 to 2017 inclusive.

In [3]:
#List of columns that we want as they are listed in CSV files. New list uses Python numbering.
csv_col_list = [1,7,19,21,36,37,40,41,45,46,47,48,49,50,51,52]
py_col_list = [i - 1 for i in csv_col_list]

#function to import a CSV file for a year into a dataframe
def CSVtoPD(year):
    filepath = "data/" + str(year) + " formatted.csv"
    
    #removes columns and rows that we don't want
    df = pd.read_csv(filepath,usecols=py_col_list)
    df = df.drop([0],axis=0)
    
    #function to tidy up the city names
    def change_name(name):
        split_name = name.split()
        return(split_name[0])
    
    #calling the above function
    df["Unnamed: 0"] = df["Unnamed: 0"].apply(change_name)

    #change column names
    df.rename(columns = {"Unnamed: 0":"City",
                         "PKW Dichte (je 1 000 Einwohner)":"PKW Dichte",
                         "Kraftfahrzeuge insgesamt":"KFZ insgesamt",
                         "Kraftfahrzeuge Dichte (je 1000 Einwohner)":"KFZ Dichte"
                         }, inplace = True)
    
    '''
    Add empty column for electric cars and puts it after the PKW diesel column
    df['PKW Elektro'] = "NaN"
    df.insert(8, 'PKW Elektro', df.pop('PKW Elektro'))
    '''
    
    #sorts the df alphabetically by city name
    df = df.sort_values(by = "City")
    
    #1-10 numbering on the index
    df = df.reset_index(drop=True)
    
    #add column with year
    df = df.assign(Year=year)
    
    return(df)

In [4]:
df2012 = CSVtoPD(2012)
df2013 = CSVtoPD(2013)
df2014 = CSVtoPD(2014)
df2015 = CSVtoPD(2015)
df2016 = CSVtoPD(2016)
df2017 = CSVtoPD(2017)

,City,PKW insgesamt,PKW Dichte,LKW insgesamt,KFZ insgesamt,KFZ Dichte,PKW Benzin,PKW Diesel,PKW Emissionsgruppe Euro 1,PKW Emissionsgruppe Euro 2,PKW Emissionsgruppe Euro 3,PKW Emissionsgruppe Euro 4,PKW Emissionsgruppe Euro 5,PKW Emissionsgruppe Euro 6,PKW Emissionsgruppe sonstige,PKW Emissionsgruppe schadstoffreduzierte insgesamt,Year
0,Berlin,1135704,328,78367,1327015,383,886562,230836,71138,253535,186186,457691,150419,536,2905,1122410,2012
1,Dortmund,258153,445,12827,296337,511,190795,63695,14176,55115,43643,106485,33815,70,1371,254675,2012
2,Düsseldorf,282437,480,14373,322919,548,181871,97015,15205,49275,42674,108426,59875,258,1544,277257,2012
3,Essen,266528,464,14795,305552,532,197831,64084,14221,53681,43474,112201,39313,113,642,263645,2012
4,Frankfurt,308146,453,20601,351326,517,188755,116062,15174,50971,43499,114906,76143,297,1177,302167,2012
5,Hamburg,731283,409,48226,841862,471,511011,213365,35477,135210,112385,385853,145503,696,4046,719170,2012
6,Köln,438952,436,25892,504710,501,302061,131396,24166,81655,71635,173014,73894,210,2479,427053,2012
7,Leipzig,200054,383,13484,226527,433,157308,39713,9747,39109,34168,87496,27419,52,532,198523,2012
8,München,663127,490,28760,755823,559,419398,238535,31742,103461,97330,237557,168693,936,7698,647417,2012
9,Stuttgart,277606,458,12446,315172,520,184664,90600,11870,50848,41675,105145,62913,543,745,273739,2012


## Making PKW dataframes from the years where we have a .xlsx

This is for all years 2018 to 2022 inclusive.
The 2018 Excel file is structured slightly differently to the other Excel files - I edited it in Excel to make it consistent so that it can use the same function.

In [5]:
#these numbers are used in the spreadsheet to identify our 10 cities
cities = "11000|02000|09162|05315|06412|08111|05111|14713|05913|05113"

#function to turn an .XLSX file for year into a df
def ExceltoDF(year):
    #generate filepath for the relevant year
    filepath = "data/fz1_" + str(year) + ".xlsx"
    
    #read sheet FZ1.1 from that file path, skip first 8 roads, skip columns without header names
    df1 = pd.read_excel(filepath, sheet_name='FZ1.1',skiprows=8,usecols=lambda x: 'Unnamed' not in x)

    #fill the merged cells with the corresponding value
    df1 = df1.fillna(method='ffill')

    #remove the final 5 rows. They don't have relevant data and are in a different format.
    df1 = df1[:-6]
    
    #same as above for most of sheet 2
    #the Excel formatting makes it hard to get the columns before Benzin but they're not needed
    df2 = pd.read_excel(filepath, sheet_name='FZ1.2',skiprows=8,usecols=lambda x: 'Unnamed' not in x)
    df2 = df2.fillna(method='ffill')
    df2 = df2[:-5]
    
    #combines the dataframes from each of the sheets
    df = pd.concat([df1, df2], axis=1)
    
    #filters the dataframe to only show rows with our 10 cities    
    df = df.dropna(subset=['Statistische Kennziffer und Zulassungsbezirk'])
    df = df.loc[df['Statistische Kennziffer und Zulassungsbezirk'].str.contains(cities)]
    
    #Berlin and Hamburg are cities and Länder so this line is needed to stop them from being duplicated in the df
    df = df.loc[df["Land"].str.contains('INSGESAMT')==False]
    
    #remove unnecessary columns - for electro add back in 40
    df = df.iloc[:,[2,8,19,20,33,34,35,36,42,43,44,45,46,47,50,51]]

    #function to remove numbers and STADT from city names, change to umlaut spelling, and make nice capitalisation
    def change_name(name):
        
        #removes STADT from the string
        name = name.replace(', STADT','')
        name = name.replace(',STADT','')
        
        #splits the string of the current name
        split_name = name.split()
        
        #changes the oe kind of umlaut, taking the correct city name from the list created by split_name
        umlauts = split_name[1].replace("OE","Ö")
        #changes the ue kind of umlaut, taking the city name from the variable created above
        umlauts = umlauts.replace("UE","Ü")
        
        #capitalises the first letter of the cityname
        capitals = umlauts.capitalize()
        
        return(capitals)
    
    #calling the above function to make names look nice
    df['Statistische Kennziffer und Zulassungsbezirk'] = df['Statistische Kennziffer und Zulassungsbezirk'].apply(change_name)
    
    #renames the columns
    df.columns = df.columns.str.replace('Euro', 'PKW Emissionsgruppe Euro')
    df.rename(columns = {'Statistische Kennziffer und Zulassungsbezirk':"City",
                         'insgesamt.1':'PKW insgesamt', 
                         'PKW-Dichte\nje 1.000 \nEinwohner':'PKW Dichte',
                         'insgesamt.2':'LKW insgesamt',
                         'insgesamt.4':'KFZ insgesamt',
                         'Kfz-Dichte\nje 1.000 \nEinwohner':'KFZ Dichte',
                         'sonstige.1':'PKW Emissionsgruppe sonstige',
                         'schadstoff-\nreduzierte \ninsgesamt':'PKW Emissionsgruppe schadstoffreduzierte insgesamt',
                         'Benzin':"PKW Benzin",
                         "Diesel":"PKW Diesel",
                         #"Elektro":"PKW Elektro",
                         #"Elektro (BEV)":"PKW Elektro"
                        }, inplace = True)
    
    #sorts the df alphabetically by city name
    df = df.sort_values(by="City")
    
    #1-10 numbering on the index
    df = df.reset_index(drop=True)
    
    #add column with year
    df = df.assign(Year=year)
    
    return(df)

In [6]:
df2018 = ExceltoDF(2018)
df2019 = ExceltoDF(2019)
df2020 = ExceltoDF(2020)
df2021 = ExceltoDF(2021)

In [7]:
df2018.head()

,City,PKW insgesamt,PKW Dichte,LKW insgesamt,KFZ insgesamt,KFZ Dichte,PKW Benzin,PKW Diesel,PKW Emissionsgruppe Euro 1,PKW Emissionsgruppe Euro 2,PKW Emissionsgruppe Euro 3,PKW Emissionsgruppe Euro 4,PKW Emissionsgruppe Euro 5,PKW Emissionsgruppe Euro 6,PKW Emissionsgruppe sonstige,PKW Emissionsgruppe schadstoffreduzierte insgesamt,Year
0,Berlin,1202829.0,342,96943.0,1422065.0,404,863140.0,307976.0,22417.0,112777.0,119537.0,366516.0,311836.0,249216.0,16951.0,1471.0,2018
1,Dortmund,282290.0,482,14019.0,322549.0,550,196743.0,80198.0,4494.0,24474.0,27396.0,88984.0,75001.0,57000.0,3963.0,258.0,2018
2,Düsseldorf,307571.0,502,16061.0,352030.0,575,183176.0,118266.0,5645.0,22568.0,24962.0,77702.0,78259.0,90912.0,6249.0,303.0,2018
3,Essen,285071.0,489,16040.0,326352.0,560,201491.0,77238.0,4320.0,23558.0,26568.0,86096.0,76435.0,63926.0,3616.0,149.0,2018
4,Frankfurt,333768.0,456,22358.0,381530.0,521,186420.0,141909.0,5159.0,22429.0,25709.0,82794.0,80938.0,108853.0,6666.0,217.0,2018


*To do with combining the PKW DFs*
1. Add 2012
2. Have exactly the same columns for the CSV-DFs and the Excel-DFs so that there aren't any NaN cells

In [13]:
#combine all dfs to one df
frames = [df2012, df2013, df2014, df2015, df2016, df2017, df2018, df2019, df2020, df2021]
df_pkw = pd.concat(frames,ignore_index=True)

#sort the df alphabetically by city name and year
df_pkw = df_pkw.sort_values(['City', 'Year'])

#reset index 
df_pkw = df_pkw.reset_index(drop=True)

#move the Year column for readability
df_pkw.insert(1, 'Year', df_pkw.pop('Year'))

#show final df
df_pkw

,City,Year,PKW insgesamt,PKW Dichte,LKW insgesamt,KFZ insgesamt,KFZ Dichte,PKW Benzin,PKW Diesel,PKW Emissionsgruppe Euro 1,PKW Emissionsgruppe Euro 2,PKW Emissionsgruppe Euro 3,PKW Emissionsgruppe Euro 4,PKW Emissionsgruppe Euro 5,PKW Emissionsgruppe Euro 6,PKW Emissionsgruppe sonstige,PKW Emissionsgruppe schadstoffreduzierte insgesamt
0,Berlin,2012,1135704,328,78367,1327015,383,886562,230836,71138,253535,186186,457691,150419,536,2905,1122410
1,Berlin,2013,1149520,328,81085,1344876,384,879752,248543,56948,227817,177838,445819,222607,2204,2621,1135854
2,Berlin,2014,1154106,342,82771,1352561,401,866100,264358,45257,199495,168162,434098,281066,9306,2430,1139814
3,Berlin,2015,1165215,341,85664,1368868,400,859426,280281,36806,174647,157555,421673,329153,28070,2488,1150392
4,Berlin,2016,1178417,340,89879,1387733,400,854691,296399,30721,151390,145941,405673,342831,83363,2699,1162618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Stuttgart,2017,298172,487,15339,341858,558,182461,111148,5161,25747,26962,79422,83244,71586,950,293072
96,Stuttgart,2018,301508.0,483,16037.0,346429.0,555,189027.0,106608.0,4624.0,22423.0,23531.0,73801.0,73009.0,97405.0,5356.0,180.0
97,Stuttgart,2019,301793.0,477,16417.0,347839.0,550,196371.0,96958.0,4101.0,19457.0,20005.0,66912.0,66242.0,117434.0,2258.0,296409.0
98,Stuttgart,2020,307367.0,484,16985.0,354454.0,558,207228.0,86990.0,3689.0,16485.0,15542.0,58288.0,60393.0,143689.0,3477.0,301563.0


## Making pollution dataframes

In [9]:
#function to turn air quality excel into a df
def Air_exceltoDF(year):
  #generate filepath for the relevant year
  filepath = 'data/data air quality/Jahresbilanzen_Feinstaub_'+str(year)+'.xlsx'

  #read excel
  df_air = pd.read_excel(filepath)

  #fill the merged cells with the corresponding value
  df_air = df_air.fillna(method='ffill')

  #filter for the 10 biggest cities
  df_air = df_air[df_air['Stationsname'].str.contains('Berlin|Hamburg|München|Köln|Frankfurt|Stuttgart|Düsseldorf|Leipzig|Dortmund|Essen')]

  #remove rows with "Leipziger Straße" or "Frankfurt(Oder)" or "Stuttgarter Straße" or "Düsseldorfer Straße"
  df_air = df_air[df_air["Stationscode"].str.contains('DEBB092|DESN083|DEBB045|DEBW128|DEBW157|DENW100') == False]

  #filter for needed variables
  df_air = df_air.loc[:,['Bundesland / Messnetz','Stationsname', 'Jahresmittelwert in µg/m³']]

  #create new columns "City"
  def change_name(name):
    name=name.replace('-',' ')
    name=name.replace('/',' ')
    split_name = name.split(' ')
    return split_name[0]
  df_air['City'] = df_air['Stationsname'].apply(change_name)

  #mean for each city
  df_air = df_air.groupby(['City']).mean()

  #translate column in english
  df_air.rename(columns={'Jahresmittelwert in µg/m³': 'Annual mean in µg/m³'}, inplace=True) 

  #sort the df alphabetically by city name
  df_air = df_air.sort_values(by='City')

  #add index-column
  df_air = df_air.reset_index()

  #add column with year
  df_air = df_air.assign(Year=year)

  return(df_air)

In [10]:
#create df for each year
df_air_2012 = Air_exceltoDF(2012)
df_air_2013 = Air_exceltoDF(2013)
df_air_2014 = Air_exceltoDF(2014)
df_air_2015 = Air_exceltoDF(2015)
df_air_2016 = Air_exceltoDF(2016)
df_air_2017 = Air_exceltoDF(2017)
df_air_2018 = Air_exceltoDF(2018)
df_air_2019 = Air_exceltoDF(2019)
df_air_2020 = Air_exceltoDF(2020)
df_air_2021 = Air_exceltoDF(2021)

In [11]:
#combine all dfs to one df
frames = [df_air_2012, df_air_2013, df_air_2014, df_air_2015, df_air_2016, df_air_2017, df_air_2018, df_air_2019, df_air_2020, df_air_2021]
df_air = pd.concat(frames,ignore_index=True)

#sort the df alphabetically by city name and year
df_air = df_air.sort_values(['City', 'Year'])

#reset index 
df_air = df_air.reset_index(drop=True)

#show final df
df_air

,City,Annual mean in µg/m³,Year
0,Berlin,23.583333,2012
1,Berlin,24.083333,2013
2,Berlin,27.090909,2014
3,Berlin,23.272727,2015
4,Berlin,23.181818,2016
...,...,...,...
95,Stuttgart,25.250000,2017
96,Stuttgart,23.000000,2018
97,Stuttgart,21.500000,2019
98,Stuttgart,19.250000,2020


In [26]:
#combine the 2 big dataframes
df = pd.concat([df_pkw, df_air["Annual mean in µg/m³"]], axis=1)

#move pollution to beginning of table
df.insert(2, 'Annual mean in µg/m³', df.pop('Annual mean in µg/m³'))



#display the main dataframe
df

,City,Year,Annual mean in µg/m³,PKW insgesamt,PKW Dichte,LKW insgesamt,KFZ insgesamt,KFZ Dichte,PKW Benzin,PKW Diesel,PKW Emissionsgruppe Euro 1,PKW Emissionsgruppe Euro 2,PKW Emissionsgruppe Euro 3,PKW Emissionsgruppe Euro 4,PKW Emissionsgruppe Euro 5,PKW Emissionsgruppe Euro 6,PKW Emissionsgruppe sonstige,PKW Emissionsgruppe schadstoffreduzierte insgesamt
0,Berlin,2012,23.583333,1135704,328,78367,1327015,383,886562,230836,71138,253535,186186,457691,150419,536,2905,1122410
1,Berlin,2013,24.083333,1149520,328,81085,1344876,384,879752,248543,56948,227817,177838,445819,222607,2204,2621,1135854
2,Berlin,2014,27.090909,1154106,342,82771,1352561,401,866100,264358,45257,199495,168162,434098,281066,9306,2430,1139814
3,Berlin,2015,23.272727,1165215,341,85664,1368868,400,859426,280281,36806,174647,157555,421673,329153,28070,2488,1150392
4,Berlin,2016,23.181818,1178417,340,89879,1387733,400,854691,296399,30721,151390,145941,405673,342831,83363,2699,1162618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Stuttgart,2017,25.250000,298172,487,15339,341858,558,182461,111148,5161,25747,26962,79422,83244,71586,950,293072
96,Stuttgart,2018,23.000000,301508.0,483,16037.0,346429.0,555,189027.0,106608.0,4624.0,22423.0,23531.0,73801.0,73009.0,97405.0,5356.0,180.0
97,Stuttgart,2019,21.500000,301793.0,477,16417.0,347839.0,550,196371.0,96958.0,4101.0,19457.0,20005.0,66912.0,66242.0,117434.0,2258.0,296409.0
98,Stuttgart,2020,19.250000,307367.0,484,16985.0,354454.0,558,207228.0,86990.0,3689.0,16485.0,15542.0,58288.0,60393.0,143689.0,3477.0,301563.0
